![](http://mymommyology.com/wp-content/uploads/2011/06/dora-the-explorer-logo1.jpg)

In [20]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#added this because it's indicated as useful but not imported
import urllib
from zipfile import ZipFile
from io import BytesIO
import os

I Definitely copy and pasted this next piece. I found it here:
http://stackoverflow.com/a/13895723

In [27]:
def reporthook(blocknum, blocksize, totalsize):
    readsofar = blocknum * blocksize
    if totalsize > 0:
        percent = readsofar * 1e2 / totalsize
        s = "\r%5.1f%% %*d / %d" % (
            percent, len(str(totalsize)), readsofar, totalsize)
        os.sys.stderr.write(s)
        if readsofar >= totalsize: # near the end
            os.sys.stderr.write("\n")
    else: # total size is unknown
        os.sys.stderr.write("read %d\n" % (readsofar,))

In [28]:
datadir = "data"
try:
    os.makedirs(datadir)
except:
    pass #directory already exists. :P

## Q1: Is gentrification the cause of rent hikes in Crown Heights, Brooklyn?

In [ ]:
#how do we answer this?

## Q2: Are new people moving in below the median income?

In [8]:
some_zip_file = "https://www.huduser.gov/portal/datasets/fmr/fmr2016p/asq_zctadata_2013_00000to19999.zip"

In [30]:
#my attempt at data caching
try:
    zipF = ZipFile(os.path.join(datadir,some_zip_file.split("/")[-1]))
except FileNotFoundError as e:
    #if zip doesn't exist, download it and try opening it again
    urllib.request.urlretrieve(some_zip_file, os.path.join(datadir,some_zip_file.split("/")[-1]), reporthook)
    zipF = ZipFile(os.path.join(datadir,some_zip_file.split("/")[-1]))
zipF.namelist()

['asq_zctadata_2013_00000to19999.xlsx']

## Q3: How much do they contribute to the taxi industry?
#### Let's make this more specific, maybe?

In [3]:
taxi_url="https://data.cityofnewyork.us/resource/2yzn-sicd.json"

In [4]:
taxi_df = pd.read_json(taxi_url)

### What does the data look like?

In [5]:
taxi_df.head()

,dropoff_datetime,dropoff_latitude,dropoff_longitude,extra,fare_amount,mta_tax,passenger_count,payment_type,pickup_datetime,pickup_latitude,pickup_longitude,rate_code,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,vendor_id
0,2015-01-06T19:06:32.000,40.727608,-73.991150,1.0,8.0,0.5,1,2,2015-01-06T18:58:18.000,40.707218,-74.004639,1,N,0.00,0.0,9.80,1.60,1
1,2015-03-10T22:43:36.000,40.759827,-73.972092,0.5,6.5,0.5,1,1,2015-03-10T22:37:24.000,40.770870,-73.962952,1,N,1.55,0.0,9.35,1.00,1
2,2015-04-21T14:36:54.000,40.754753,-73.977829,0.0,8.5,0.5,1,1,2015-04-21T14:25:29.000,40.749969,-73.991432,1,N,2.75,0.0,12.05,1.30,1
3,2015-03-02T18:13:08.000,40.746220,-73.985039,1.0,13.0,0.5,1,2,2015-03-02T17:55:05.000,40.769695,-73.964905,1,N,0.00,0.0,14.80,2.20,1
4,2015-01-26T12:02:13.000,40.735851,-73.993103,0.0,8.5,0.5,4,1,2015-01-26T11:52:13.000,40.733055,-73.975365,1,N,1.86,0.0,11.16,1.42,2
